**17_MAW_ii**

In [1]:
import os ; import pathlib as pl ; import flopy ; import matplotlib.pyplot as plt ; import numpy as np; from pathlib import Path  
from flopy.plot.styles import styles ; import shapefile as shp ; import matplotlib as mpl
import pandas as pd ; from modflow_devtools.misc import get_env, timed ; write=get_env("WRITE", True);run=get_env("RUN", True)

In [2]:
sim_ws         = Path("./17_MAW_ii")                        ; sim_ws.mkdir(exist_ok=True) ; sim_name = "ex-gwf-maw-p02"
exe_name       = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf6.exe'    ; workspace = figs_path = sim_ws ; gwf_name = sim_name ; output_folder = sim_ws

In [8]:
length_units = "meters" ; time_units = "days" ; nper = 1 ;nlay = 2 ;nrow = 101 ;ncol = 101 ;delr = 142 ;delc = 142 ;top = -50 ;
botm_str = "-142.9,-514.5" ;strt_str = "3.05,9.14" ;k11 = 1 ;k33 = 0.0000000000000001 ;ss = 0.0001 ;maw_radius = 0.15 ;maw_rate = 0 ;
nouter = 500 ;ninner = 100 ;hclose = 0.000000001 ;rclose = 0.0001 ;

botm = [float(value) for value in botm_str.split(",")]                                      ; strt = [float(value) for value in strt_str.split(",")]
tdis_ds = ((2.314815, 50, 1.2),)        ; extents = (0.0, delr * ncol, 0.0, delc * nrow)    ; shape2d = (nrow, ncol)  ; shape3d = (nlay, nrow, ncol)
idomain = np.ones(shape3d, dtype=float) ; xw, yw = (ncol / 2) * delr, (nrow / 2) * delc     ; y = 0.0
for i in range(nrow):
    x = 0.0 ; y = (float(i) + 0.5) * delc
    for j in range(ncol):
        x = (float(j) + 0.5) * delr  ; r = np.sqrt((x - xw) ** 2.0 + (y - yw) ** 2.0)
        if r > 7163.0: idomain[:, i, j] = 0
maw_row = int(nrow / 2)  ; maw_col = int(ncol / 2)   ; maw_packagedata = [[0, maw_radius, botm[-1], strt[-1], "SPECIFIED", 2]]
maw_conn = [[0, 0, 0, maw_row, maw_col, top, botm[-1], 111.3763, -999.0],[0, 1, 1, maw_row, maw_col, top, botm[-1], 445.9849, -999.0]]
maw_spd = [[0, "rate", maw_rate], [0, "flowing_well", 0.0, 7500.0, 0.5]]

In [9]:
def build_models():
    obs_file = f"{sim_name}.maw.obs" ; csv_file = obs_file + ".csv"
    obs_dict = {csv_file: [("head", "head", (0,)),("Q1", "maw", (0,), (0,)),("Q2", "maw", (0,), (1,)),("FW", "fw-rate", (0,))]}

    sim_ws = os.path.join(workspace,sim_name)
    sim = flopy.mf6.MFSimulation    (sim_name=sim_name,sim_ws=sim_ws,exe_name="mf6")
    flopy.mf6.ModflowTdis   (sim,nper=nper,perioddata=tdis_ds,time_units=time_units)
    flopy.mf6.ModflowIms    (sim,print_option="summary",outer_maximum=nouter,outer_dvclose=hclose,inner_maximum=ninner,inner_dvclose=hclose,rcloserecord=f"{rclose}strict")
    gwf = flopy.mf6.ModflowGwf      (sim,modelname=sim_name,save_flows=True)
    flopy.mf6.ModflowGwfdis     (gwf,length_units=length_units,nlay=nlay,nrow=nrow,ncol=ncol,delr=delr,delc=delc,top=top,botm=botm,idomain=idomain)
    flopy.mf6.ModflowGwfnpf     (gwf,icelltype=0,k=k11,k33=k33,save_specific_discharge=True)
    flopy.mf6.ModflowGwfsto     (gwf,iconvert=0,ss=ss)
    flopy.mf6.ModflowGwfic      (gwf,strt=strt)
    maw=flopy.mf6.ModflowGwfmaw (gwf,flowing_wells=True,nmawwells=1,packagedata=maw_packagedata,connectiondata=maw_conn,perioddata=maw_spd)
    maw.obs.initialize (filename= obs_file, digits=10, print_input=True, continuous=obs_dict)
    flopy.mf6.ModflowGwfoc      (gwf, printrecord=[("BUDGET", "LAST")])
    return sim
def write_models(sim, silent=True):
    sim.write_simulation(silent=silent)
@timed
def run_models(sim, silent=True):
    success, buff = sim.run_simulation(silent=silent)  ; assert success, buff
def scenario(silent=True):
    sim = build_models()
    if write:  write_models(sim, silent=silent)
    if run:    run_models  (sim, silent=silent)
scenario()

run_models took 2589.56 ms
